In [1]:
# pip install selenium

In [2]:
# pip install chromedriver-autoinstaller

In [3]:
# pip install BeautifulSoup4

In [4]:
# pip install tqdm

In [5]:
# pip install urlopen

In [6]:
# pip install requests

In [7]:
import os
from PIL import Image
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import requests
from pprint import pprint

In [8]:
class Crawling:
    def __init__(self):
        self.driver = None
        self.data = []
        self.i = 1
        self.folder_path = "./whos_who"
        
    def install_driver(self):  # 드라이버 연결
        chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
        s = Service(f'./{chrome_ver}/chromedriver.exe')

        try:
            self.driver = webdriver.Chrome(service=s)  # 드라이버있으면 로드
        except:
            chromedriver_autoinstaller.install(True)  # 없으면 설치 후 로드
            self.driver = webdriver.Chrome(service=s)

        self.driver.implicitly_wait(10)
        
    def init_page(self):  # 초기 진입
        url = "https://cdb.chosun.com/search/db-people/i_common/index_s.jsp"
        self.driver.get(url)

        radio_button_xpath = "//input[@type='radio' and @value='연예인']"
        radio_button = self.driver.find_element(By.XPATH, radio_button_xpath)
        radio_button.click()

        search_button_xpath = "//button[@class='more' and contains(text(),'검색')]"
        search_button = self.driver.find_element(By.XPATH, search_button_xpath)
        search_button.click()
        
    def download_image(self, url, local_path):
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(local_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=1024):
                    file.write(chunk)
    
    def check_folder(self):
        if os.path.exists(self.folder_path):
            # 폴더 내용 삭제
            for file_name in os.listdir(self.folder_path):
                file_path = os.path.join(self.folder_path, file_name)
                try:
                    if os.path.isfile(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        os.rmdir(file_path)
                except Exception as e:
                    print(f"Fail deleting file and folder {file_path}: {e}")
        else:
            # 폴더가 없으면 생성
            os.makedirs(photo_folder_path)
                    
    def crawling(self):
        for page_number in tqdm(range(1, 21),desc="Loding.."):  # 400명 정보
        # 현재 페이지에서 정보 추출
            bsObject = BeautifulSoup(self.driver.page_source, "html.parser")
            Tags = bsObject.select("li > div.people2")

            for Tag in Tags:
                name = Tag.findChild("dl").findChild("dt").findChild("a").get_text().rstrip()
                portrait = Tag.findChild("div",{"class":"thum"}).findChild("a").get("href").split("'")[1].split("-")[1]
                self.data.append((name, portrait))

                # 이미지 다운로드 및 로컬에 저장
                image_url = f"https://cdb.chosun.com/search/db-people/i_common/manImage.jsp?f={portrait}"
                local_image_path = os.path.join(self.folder_path, f"{self.i}_{name.replace(' ', '_')}.jpg")
                self.download_image(image_url, local_image_path)

                self.i += 1

            if page_number % 10 == 0:  # 다음 페이지네이션
                next_page_button_xpath = "//a[@class='next']"
                try:
                    next_page_button = self.driver.find_element(By.XPATH, next_page_button_xpath)
                    next_page_button.click()
                except Exception as e:
                    print(f"Fail clicking next page button: {e}")
            else:
                # 다음 페이지 번호로 이동
                next_page_number_js = f"fnClickPageNavigation('{page_number + 1}');"
                self.driver.execute_script(next_page_number_js)

        self.driver.quit()  # 드라이브 종료
        
    def run(self):
        self.install_driver()
        self.init_page()
        self.check_folder()
        self.crawling()

In [9]:
task = Crawling()
task.run()

Loding..: 100%|████████████████████████████████████████████████████████████████████████| 20/20 [00:54<00:00,  2.73s/it]


In [10]:
for i, name in enumerate(task.data, 1):
    print(i, name)

1 ('남궁원', '02141999110500030828')
2 ('안성기', '02141999110500023937')
3 ('조용필', '02141999110500024071')
4 ('김창완', '02141999110500000880')
5 ('정혜선', '02142000052400215559')
6 ('이순재', '02141999110500020384')
7 ('조영남', '02141999110500024070')
8 ('전헌태', '02112010053100460600')
9 ('최불암', '02141999110500024087')
10 ('유인촌', '02141999110500023963')
11 ('정동환', '02141999110500024049')
12 ('이병헌', '02141999120700210917')
13 ('김동완', '02142001012000221964')
14 ('이정재', '02141999110500030830')
15 ('조혜련', '02141999110500027973')
16 ('최수종', '02141999110500024089')
17 ('김혜수', '02141999110500027899')
18 ('신구', '02141999110500030854')
19 ('윤여정', '02141999110500024942')
20 ('이효리', '02141999122000211267')
21 ('고두심', '02141999110500024453')
22 ('인순이', '02141999110500006611')
23 ('송현', '02141999110500009493')
24 ('신영균', '02141999110500005782')
25 ('정보석', '02141999110500024051')
26 ('조재현', '02141999110500032561')
27 ('김영애', '02141999110500024482')
28 ('전진', '02142001011900221958')
29 ('정영숙', '02141999110500030825